# SF Bike Share

## Overview

## Load data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pprint
%matplotlib inline

data = pd.read_csv('bikeshare_trips.csv')

In [2]:
data[0:5]

,trip_id,duration_sec,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,zip_code,subscriber_type
0,650330,495,2015-02-19 15:46:00 UTC,Yerba Buena Center of the Arts (3rd @ Howard),68,2015-02-19 15:54:00 UTC,San Francisco Caltrain 2 (330 Townsend),69,463,94061,Subscriber
1,367544,241,2014-07-17 08:09:00 UTC,San Francisco Caltrain 2 (330 Townsend),69,2014-07-17 08:13:00 UTC,Townsend at 7th,65,441,95120,Subscriber
2,586344,3589,2014-12-27 16:01:00 UTC,Embarcadero at Bryant,54,2014-12-27 17:01:00 UTC,2nd at Townsend,61,466,27401,Customer
3,772874,733,2015-05-19 09:50:00 UTC,Davis at Jackson,42,2015-05-19 10:02:00 UTC,2nd at Townsend,61,501,94111,Subscriber
4,419431,302,2014-08-21 19:11:00 UTC,5th at Howard,57,2014-08-21 19:16:00 UTC,San Francisco Caltrain (Townsend at 4th),70,392,95110,Subscriber


In [3]:
len(data.start_station_name.unique())

84

In [4]:
len(data.end_station_name.unique())

84

In [5]:
len(data.bike_number.unique())

700

In [6]:
len(data)

983648

## Format data and time

In [7]:
# Remove UTC from datatime !!Don't do this more than once!!
data['start_date'] = data['start_date'].astype(str).str[:-3]

In [8]:
data['start_date'] = pd.to_datetime(data['start_date'])

In [10]:
data['end_date'] = pd.to_datetime(data['end_date'])

In [ ]:
data[0:5]

In [ ]:
len(data.start_date.dt.year.unique())

In [ ]:
len(data.start_date.dt.month.unique())

In [15]:
data['year'] = data.start_date.dt.year

In [16]:
data['month'] = data.start_date.dt.month

In [17]:
data['start_day'] = data.start_date.dt.day

In [18]:
data['start_time'] = data.start_date.dt.time

In [19]:
data['end_day'] = data.end_date.dt.day

In [20]:
data['end_time'] = data.end_date.dt.time

In [ ]:
data[0:5]

In [23]:
data['trip_length'] = data['end_date'] - data['start_date']

In [34]:
data['trip_length'][0:10]

0   00:08:00
1   00:04:00
2   01:00:00
3   00:12:00
4   00:05:00
5   00:05:00
6   00:20:00
7   00:06:00
8   00:15:00
9   00:07:00
Name: trip_length, dtype: timedelta64[ns]

In [25]:
max(data['trip_length'])

Timedelta('199 days 22:19:00')

In [33]:
# Remove trips longer than 3 days 
np.mean(data['trip_length'])

Timedelta('0 days 00:16:58.983782')